The nonlinear diffusion algorithm takes three parameters: $\lambda, \kappa$, and the number of iterations.


In [15]:
using Pkg
Pkg.activate("calval")
Pkg.resolve()
using IceFloeTracker
using IceFloeTracker: strel_diamond
using Images
using Images: test_approx_eq_sigma_eps
include("../scripts/dev/nonlinear_diffusion.jl")

  Activating project at `~/Documents/research/calval_tgrs/notebooks/calval`
  No Changes to `~/Documents/research/calval_tgrs/notebooks/calval/Project.toml`
  No Changes to `~/Documents/research/calval_tgrs/notebooks/calval/Manifest.toml`


anisotropic_diffusion_2D (generic function with 1 method)

In [47]:
struct_elem2 = strel_diamond((5, 5)) #original matlab structuring element -  a disk-shaped kernel with radius of 2 px
test_region = (1:2707, 1:4458)
test_data_dir = "/Users/dmw/Documents/research/IceFloeTracker.jl/test"
matlab_normalized_img_file = "$(test_data_dir)/test_inputs/matlab_normalized.png"
matlab_sharpened_file = "$(test_data_dir)/test_inputs/matlab_sharpened.png"
matlab_diffused_file = "$(test_data_dir)/test_inputs/matlab_diffused.png"
matlab_equalized_file = "$(test_data_dir)/test_inputs/matlab_equalized.png"
landmask_bitmatrix = convert(BitMatrix, float64.(load("$(test_data_dir)/test_inputs/current_landmask.png")))
landmask_no_dilate = convert(BitMatrix, float64.(load("$(test_data_dir)/test_inputs/landmask_no_dilate.png")))
input_image = float64.(load("$(test_data_dir)/test_inputs/beaufort-chukchi-seas_truecolor.2020162.aqua.250m.tiff")[test_region...])
matlab_norm_image = float64.(load(matlab_normalized_img_file)[test_region...])
matlab_sharpened = float64.(load(matlab_sharpened_file))
matlab_diffused = float64.(load(matlab_diffused_file)[test_region...])
matlab_equalized = float64.(load(matlab_equalized_file))
nothing

In [38]:
n_iters = 3
kappa = 0.1
lambda = 0.1

julia_input_landmasked = IceFloeTracker.apply_landmask(input_image, landmask_no_dilate);

In [44]:
# The default function type is "exponential"
# The default number of iterations in the matlab code is 5.
lambda = 0.1
kappa = 0.1
julia_diffused = deepcopy(julia_input_landmasked)
for n_iters in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    julia_diffused .= nonlinear_diffusion(deepcopy(julia_input_landmasked), PeronaMalikDiffusion(lambda, kappa, n_iters, "exponential"))
    print(n_iters, " ", test_approx_eq_sigma_eps(julia_diffused, matlab_diffused, ones(2), 0.01, true), "\n")
end

1 0.004649233581938792
2 0.004297235115938635
3 0.004029358776186884
4 0.003850935844348241
5 0.0037702901392723874
6 0.003778670850896731
7 0.0038525085008467236
8 0.003968538167231018
9 0.004109797329568576
10 0.004265346587725617


In [45]:
# The default value for the gradient scaling is estimated from the image histogram.
n_iters = 5
for kappa in [0.05, 0.1, 0.15, 0.25]
    julia_diffused .= nonlinear_diffusion(deepcopy(julia_input_landmasked), PeronaMalikDiffusion(lambda, kappa, n_iters, "exponential"))
    print(kappa, " ", test_approx_eq_sigma_eps(julia_diffused, matlab_diffused, ones(2), 0.01, true), "\n")
end

0.05 0.00395704836316505
0.1 0.0037702901392723874
0.15 0.003853526312136462
0.25 0.004055243056540083


In [46]:
n_iters = 5
kappa = 0.1
for lambda in [0.05, 0.1, 0.15, 0.25]
    julia_diffused .= nonlinear_diffusion(deepcopy(julia_input_landmasked), PeronaMalikDiffusion(lambda, kappa, n_iters, "exponential"))
    print(lambda, " ", test_approx_eq_sigma_eps(julia_diffused, matlab_diffused, ones(2), 0.01, true), "\n")
end

0.05 0.00403681636764216
0.1 0.0037702901392723874
0.15 0.0038595395485302698
0.25 0.004485201576206776
